<a href="https://colab.research.google.com/github/silverstar0727/1day-1commit-challenge/blob/master/machine_learning_ch8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## IMBDb영화 리뷰

In [0]:
!pip install pyprind

In [0]:
import pyprind
import pandas as pd
import os

basepath = 'C:\\Users\\silve\\Desktop\\aclImdb_v1\\aclImdb'

labels = {'pos': 1, 'neg': 0}
pbar = pyprind.ProgBar(50000)

df = pd.DataFrame()
for s in('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in sorted(os.listdir(path)):
            with open(os.path.join(path, file), 'r', encoding = 'utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], ignore_index = True)
            pbar.update()
            
df.columns = ['review', 'sentiment']

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:02:03


In [0]:
import numpy as np

np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('movie_data.csv', index = False, encoding = 'utf-8')

In [0]:
df = pd.read_csv('movie_data.csv', encoding = 'utf-8')
df.head()

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0


In [0]:
df.shape

(50000, 2)

### BOW model

In [0]:
# BoW 모델을 만듦
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
docs = np.array(['The sun is shining', 'The weather is sweet', 
                 'The sun is shining, the weater is sweet, and one and one is two'])

bag = count.fit_transform(docs)

CountVetorizer모듈에 내장되어 있는 vocabulary메소드 사용

단어와 정수를 매핑하여 딕셔너리형태로 저장.

In [0]:
print(count.vocabulary_)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 9, 'sweet': 5, 'weater': 8, 'and': 0, 'one': 2, 'two': 7}


In [0]:
print(bag.toarray())

[[0 1 0 1 1 0 1 0 0 0]
 [0 1 0 0 0 1 1 0 0 1]
 [2 3 2 1 1 1 2 1 1 0]]


## 단어의 적합성 평가
tf-idf(term frequency-inverse document frequency)

In [0]:
# 사이킷런의 TfidTransFormer클래스 사용
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf = True, norm = 'l2', smooth_idf = True)
np.set_printoptions(precision = 2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.   0.  ]
 [0.   0.39 0.   0.   0.   0.5  0.39 0.   0.   0.66]
 [0.5  0.44 0.5  0.19 0.19 0.19 0.29 0.25 0.25 0.  ]]


In [0]:
df.loc[0, 'review'][-50:]

'is seven.<br /><br />Title (Brazil): Not Available'

In [0]:
# 텍스트 데이터 정제
import re

def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = (re.sub('[\W]+',' ', text.lower()) + ' '.join(emoticons).replace('-',''))
    return text

In [0]:
preprocessor(df.loc[0, 'review'][-50:])

'is seven title brazil not available'

In [0]:
preprocessor("<\a>This :) is :( a test :-)!")

'this is a test :) :( :)'

In [0]:
# 문서를 토큰으로 나누기
def tokenizer(text):
    return text.split()

tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [0]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]
tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [0]:
# 불용어집합을 다운로드
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\silve\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
from nltk.corpus import stopwords

stop = stopwords.words('english')

[w for w in tokenizer_porter('a runner likes running and runs a lot')[-10:] if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

In [0]:
# 문서분류를 위한 로지스틱 회귀 모델 훈련
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[:25000, 'review'].values
y_test = df.loc[:25000, 'sentiment'].values

In [0]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents = None, lowercase = False, preprocessor = None)
param_grid = [{'vect__ngram_range':[(1,1)],
              'vect__stop_word':[stop, None],
              'vect__tokenizer':[tokenizer, tokenizer_porter],
              'clf__penalty': ['l1','l2'],
              'clf__C':[1.0, 10.0, 100.0]},
             {'vect__ngram_range': [(1,1)],
             'vect__stop_words': [stop, None],
             'vect__tokenizer': [tokenizer, tokenizer_porter],
             'vect__use_if':[False],
             'vect__norm':[None],
             'clf__penalty':['l1','l2'],
             'clf__C':[1.0, 10.0, 100.0]}
             ]
lr_tfidf = Pipeline([('vect', tfidf),
                    ('clf', LogisticRegression(solver = 'liblinear', random_state = 0))])
gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid, scoring = 'accuracy', cv = 5, verbose = 1, n_jobs = -1)
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


ValueError: Invalid parameter stop_word for estimator TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=False, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None). Check the list of available parameters with `estimator.get_params().keys()`.